This is a notebook for a simple DMRG algorithm: Modern MPS Version

When writing the julia code shown in this notebook, I refer to Mr.Min Long's python code. 

https://colab.research.google.com/github/DavidGoing/DMRG/blob/main/DMRG_MPS.ipynb

We take 1D Heisenberg model as an example, show the underlying code of MPS version DMRG.

We DO NOT use good quantum numbers in the system.

In [4]:
using Einsum
using TensorOperations
using KrylovKit
using LinearAlgebra
#initial E and F for the left and right vacuum states
function initial_E(W)
    E = zeros(1,size(W,1),1)
    E[1,1,1] = 1
    return E
end
function initial_F(W)
    F = zeros(1,size(W,3),1)
    F[1,end,1] = 1
    return F
end
# MPS-MPO-MPS contract from left
# here we use the macro: @einsum A[i, j] := B[i, k] * C[k, j]
function contract_from_left(W,A,E,B)
    @einsum tensor1[a,k,s,j] := E[i,a,k]*A[i,s,j]
    @einsum tensor2[k,j,b,t] := tensor1[a,k,s,j]*W[a,s,b,t]
    @einsum tensor3[j,b,l] := tensor2[k,j,b,t]*B[k,t,l]
    return tensor3
end
# MPS-MPO-MPS contract from right
function contract_from_right(W,A,F,B)
    @einsum tensor1[i,s,b,l] := F[j,b,l]*A[i,s,j]
    @einsum tensor2[i,a,t,l] := tensor1[i,s,b,l]*W[a,s,b,t]
    @einsum tensor3[i,a,k] := tensor2[i,a,t,l]*B[k,t,l]
    return tensor3
end
#construct F lib
# MPO is the W chain
# Alist and Blist are the MPS: the chain of A and B
# MF is the right boundary of F
function construct_F(Alist,MPO,Blist)
    MF = initial_F(MPO[end])
    F = [MF]
    for i = length(MPO):-1:2
        f = contract_from_right(MPO[i],Alist[i],F[1],Blist[i])
        F = vcat([f],F)
    end
    return F
end
#construct E lib
#ME is the left boundary of E
function construct_E(Alist,MPO,Blist)
    ME = initial_E(MPO[1])
    E = [ME]
    return E
end
# calculate expectation value of MPS
function Expectation(Alist,MPO,Blist)
    E = initial_E(MPO[1])
    for i = 1:length(MPO)
        E = contract_from_left(MPO[i],Alist[i],E,Blist[i])
    end
    return E[1,1,1]
end
# coarse-graining of two site MPO into one site
function coarse_grain_MPO(W1,W2)
    @einsum W[a,s,u,c,t,v] := W1[a,s,b,t]*W2[b,u,c,v]
    ts = size(W) #tensor size
    W12 = reshape(W,(ts[1],ts[2]*ts[3],ts[4],ts[5]*ts[6]))
    return W12
end
# product of two MPO sites
function product_W(W1,W2)
    @einsum W[a,c,s,b,d,u] := W1[a,s,b,t]*W2[c,t,d,u]
    ts = size(W) #tensor size
    W12 = reshape(W,(ts[1]*ts[2],ts[3],ts[4]*ts[5],ts[6]))
    return W12
end
# product of two Many-Body MPO 
function product_MPO(MPO1,MPO2)
    @assert length(MPO1)==length(MPO2)
    result = []
    for i = 1:length(MPO1)
        result = vcat(result,[product_W(MPO1[i],MPO2[i])])
    end
    return result
end

# coarse-graining of two-site MPS in to one site
function coarse_grain_MPS(A1,A2)
    @einsum A12[i,s,t,k] := A1[i,s,j]*A2[j,t,k]
    ts = size(A12) #tensor size
    A12 = reshape(A12,(ts[1],ts[2]*ts[3],ts[4]))
    return A12
end
#reversive process of MPS coarse graining: SVD factorize
function fine_grain_MPS(A,dims)
    @assert size(A,2) == dims[1]*dims[2]
    ts = size(A) #tensor size
    Theta = reshape(A,(ts[1],dims[1],dims[2],ts[3]))
    @tensor Theta[s,i,t,k] := Theta[i,s,t,k]
    Theta = reshape(Theta,(ts[1]*dims[1],dims[2]*ts[3]))
    F  = svd(Theta)
    bond1 = length(F.U)/dims[1]/ts[1]
    bond2 = length(F.Vt)/dims[2]/ts[3]
    bond1 = convert(Int,bond1)
    bond2 = convert(Int,bond2)
    U = reshape(F.U,(dims[1],ts[1],bond1))
    Vt = reshape(F.Vt,(bond2,dims[2],ts[3]))
    S = F.S
    return U,S,Vt
end
# U,S,V truncation
function truncate_MPS(U,S,Vt,m)
    m = min(length(S),m)
    trunc = sum(S[m+1:end])
    S = S[1:m]
    U = U[:,:,1:m]
    Vt = Vt[1:m,:,:]
    S = S/norm(S)
    return U,S,Vt,trunc,m
end
# get the ProjMPO
function HamiltonianMultiply(E,W,F,A)
    @einsum tensor1[j,s,a,k] := E[i,a,k]*A[i,s,j]
    @einsum tensor2[j,b,t,k] := tensor1[j,s,a,k]*W[a,s,b,t]
    @einsum R[k,t,l] :=tensor2[j,b,t,k]*F[j,b,l]
    return R
end

function optimize_two_sites(A1,A2,W1,W2,E,F,m,direction)
    W = coarse_grain_MPO(W1,W2)
    A0 = coarse_grain_MPS(A1,A2)
    H_eff(A) = HamiltonianMultiply(E,W,F,A)
    Eig,AA,info = eigsolve(H_eff, A0, 1, :SR)
    U,S,Vt = fine_grain_MPS(AA[1],[size(A1,2),size(A2,2)])
    U,S,Vt,trunc,m = truncate_MPS(U,S,Vt,m)
    s_diag = diagm(S)
    if direction== "right"
        @einsum A2[i,t,k] := s_diag[i,j]*Vt[j,t,k]
        @tensor U[i,s,j] := U[s,i,j]
        return Eig[1],U,A2,trunc,m
    else
        @assert direction =="left"
        @einsum A1[i,s,j] := U[s,i,m]*s_diag[m,j]
        return Eig[1],A1,Vt,trunc,m
    end
end

function two_site_dmrg(MPS,MPO,m,sweeps)
    E = construct_E(MPS,MPO,MPS)
    F = construct_F(MPS,MPO,MPS)
    deleteat!(F,1)
    for sweep = 1:(sweeps÷2)
        for i = 1:length(MPS)-2
            Energy,MPS[i],MPS[i+1],trunc,states = optimize_two_sites(MPS[i],MPS[i+1],MPO[i],MPO[i+1],E[end],F[1],m,"right")
            println("Sweep $(sweep*2)  Sites $i $(i+1) Eenrgy $Energy states $states trunc $trunc")
            e = contract_from_left(MPO[i],MPS[i],E[end],conj(MPS[i]))
            E = vcat(E,[e])
            deleteat!(F,1)
        end
        for i = length(MPS)-1:-1:2
            Energy,MPS[i],MPS[i+1],trunc,states = optimize_two_sites(MPS[i],MPS[i+1],MPO[i],MPO[i+1],E[end],F[1],m,"left")
            println("Sweep $(sweep*2+1) Sites $i $(i+1) Energy $Energy states $states trunc $trunc")
            f = contract_from_right(MPO[i+1],MPS[i+1],F[1],conj(MPS[i+1]))
            F = vcat([f],F)
            deleteat!(E,length(E))
        end
    end
    return MPS
end
            
                

two_site_dmrg (generic function with 1 method)

In [5]:
d = 2 # local bond dimenstion
N = 100 #number of sites
D = 10 #virtue bond dimenstion
InitialA1 = zeros(1,d,1)
InitialA1[1,1,1] = 1
InitialA2 = zeros(1,d,1)
InitialA2[1,2,1] = 1
#Initial State |010101010101>
MPS = repeat([InitialA1,InitialA2],convert(Int,N/2))
I = diagm([1,1])
Z = zeros(2,2)
Sz = [0.5 0;0 -0.5]
Sp = [0 0; 1 0]
Sm = [0 1; 0 0]
# define MPO element( except the element on two boundaries)
W = zeros(5,5,2,2) #[a,b,s,t]
W[1,1,:,:] = I
W[1,2,:,:] = Sz
W[1,3,:,:] = 0.5*Sp
W[1,4,:,:] = 0.5*Sm
W[2,5,:,:] = Sz
W[3,5,:,:] = Sm
W[4,5,:,:] = Sp
W[5,5,:,:] = I
@tensor W[a,s,b,t] := W[a,b,s,t]
# define the first element of MPO
Wfirst = zeros(1,5,2,2) #[a,b,s,t]
Wfirst[1,1,:,:] = I
Wfirst[1,2,:,:] = Sz
Wfirst[1,3,:,:] = 0.5*Sp
Wfirst[1,4,:,:] = 0.5*Sm
@tensor Wfirst[a,s,b,t] := Wfirst[a,b,s,t]
# define the last element of MPO
Wlast = zeros(5,1,2,2) #[a,b,s,t]
Wlast[2,1,:,:] = Sz
Wlast[3,1,:,:] = Sm
Wlast[4,1,:,:] = Sp
Wlast[5,1,:,:] = I
@tensor Wlast[a,s,b,t] := Wlast[a,b,s,t]
# construct MPO chain
MPO = vcat([Wfirst],repeat([W],N-2),[Wlast]);
HamSquared = product_MPO(MPO,MPO)
MPS = two_site_dmrg(MPS,MPO,D,8)
Energy = Expectation(MPS,MPO,conj(MPS))
println("Final energy expectation value $Energy")
H2 = Expectation(MPS,HamSquared,MPS)
println("variance = $(H2-Energy*Energy)")

1946.2462776646785